### 抓取雪球发帖

- 作者：DGC'Idea
- 版本：V0.1   
- 更新日期：2018年4月14日

----------

- 研究目的：获取微信公众号文章；

In [1]:
#引入包
import pandas as pd
import requests
import re
import json
import time
import os


# 自定义
import sys
sys.path.append('/home/jquser')

from dm import *
from tl import exists_file

# 数据目录
_data_path='../../Data/'
# 数据引擎
_data=Csv(_data_path)



#时间戳转换成日期
def timestamp_date(timestamp):
    return time.strftime('%Y-%m-%d %H:%M',time.localtime(int(str(timestamp)[0:10])))


#读取数据        
def read_xueqiu_data(user_id):
    #文件名
    data_file='xqft_%s.csv'%(user_id)
    #从cvs文件恢复数据
    df=pd.read_csv(data_file,index_col=0,parse_dates=False)
    return df


#检测文件      
def exists_xueqiu_file(user_id,data_root=_data_path):
    #cvs文件名
    data_file='%s%s.csv'%(data_root,'msg_xqft_'+user_id)
    return exists_file(data_file)


#读取数据        
def read_xueqiu_data(user_id):
    #文件名
    data_file='msg_xqft_'+user_id
    #从cvs文件恢复数据
    df=_data.read(data_file)
    return df


#保存数据        
def save_xueqiu_data(user_id,df):
    #文件名
    data_file='msg_xqft_'+user_id
    #保存到cvs文件
    _data.save(data_file,df,append=False)

    
#性别
gender={'m':'男','w':'女'}


#雪球网址
base_url='https://xueqiu.com'


#headers
header={
    "HOST":'xueqiu.com',
    'User-Agent':'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.104 \
Safari/537.36 Core/1.53.4620.400 QQBrowser/9.7.13014.400'
    }

#获取雪球用户信息
def get_xueqiu_user(user_id,cookies_str):
    #cookies_str转换为字典格式
    cookies=get_xueqiu_cookies(cookies_str)  
    url='https://xueqiu.com/v4/statuses/user_timeline.json?page=%s&user_id=%s'%(1,user_id)
    #获取数据
    r=requests.get(url,cookies=cookies,headers=header)
    #解析为json
    statuses=r.json().get('statuses')[0].get('user')
    #获得用户信息
    return (statuses.get('screen_name'),#名称
            statuses.get('province'),#城市
            gender[statuses.get('gender')],#性别
            statuses.get('status_count'),#贴子数
            statuses.get('stocks_count'),#组合数
            statuses.get('friends_count'),#关注数
            statuses.get('followers_count'),#粉丝数
            statuses.get('description'),)#简介
    
    
#爬取微信公众号文章
#user_id：用户识别码
#cookies_str：登陆后的cookies字符串，每次登陆后都不一样，所以要手动获取
#start：更新的起始位置，用以实现断点续爬和增量更新
def get_xueqiu_link(user_id,cookies_str,start_date,first=True,start=0):
    #cookies_str转换为字典格式
    cookies=get_xueqiu_cookies(cookies_str)  
    #数据列表
    data_list=[]
    #当前页
    page=1
    #页面总数
    maxpage=1
    #计数器
    count=1
    #捕获错误
    try:
        #遍历所有页
        while page<=maxpage:
            #url
            url='https://xueqiu.com/v4/statuses/user_timeline.json?page=%s&user_id=%s'%(page,user_id)
            #获取数据
            r=requests.get(url,cookies=cookies,headers=header)
#             print (r)
            #解析为json
            statuses=r.json()
            #获得贴子信息
            item_list=statuses.get('statuses')
            #如果无数据跳出循环
            if len(item_list)==0:
                break
            #如果是首页，获取总页数    
            if page==1:
                maxpage=int(statuses.get('maxPage')) 
            #遍历当前页文章列表
            for item in item_list:
                #剔除置顶贴
                if count>1:
                    #贴子发表时间
                    created_at=timestamp_date(item.get('created_at'))
                    #如果是首次获取
                    if first==True:
                        #获取早于当前时间的所有贴子
                        if created_at<start_date :
                            #文章链接、标题、时间戳（时间戳转换为日期）
                            data_list.append([created_at,item.get('title'),item.get('description'),item.get('text'),
                                              base_url+item.get('target'),False])
                    #增量获取
                    else:
                        #获取晚于更新时间的所有贴子
                        if created_at>start_date:
                             #文章链接、标题、时间戳（时间戳转换为日期）
                            data_list.append([created_at,item.get('title'),item.get('description'),item.get('text'),
                                              base_url+item.get('target'),False])
                        else:
                            #超过更新时间，结束
                            page=maxpage+1
                            break  
                print(' 正在检索：%s 篇'%(count),end='\r')            
                #计数            
                count+=1    
            #页面数递增    
            page+=1
            #延时，防止被踢
            #time.sleep(1)
    #打印错误，并忽略        
    except Exception as e:
        print('错误：%s'%(e))
        pass
    #没取到数据    
    if len(data_list)==0:
        return None
    else:
        #整理数据    
        df=pd.DataFrame(data_list,columns=['date','title','description','text','link','sended'])    
        #返回数据
        return df
    
    
xueqiu_list=[
    {
    #似乎雪球的cookies可以长期使用，待验证
    'cookies':'aliyungf_tc=AQAAAGwSGAyaoQgAUHWuPLg/sqB3ejy4; device_id=dae70533361df97e2a154eb2f1af4151; __utmt=1; remember=1; remember.sig=K4F3faYzmVuqC0iXIERCQf55g2Y; xq_a_token=5654fd39cb89fef36765cc9038bc5b47ad0e62ba; xq_a_token.sig=MZOhC2VaLNqrwpqJk0ONVR8kUMY; xq_r_token=01ee39b512f28c6f395e2d5fc1eeed662ade6a39; xq_r_token.sig=U3jfCcO3T4tjqSenYk1Oevgpufg; xq_is_login=1; xq_is_login.sig=J3LxgPVPUzbBg3Kee_PquUfih7Q; u=6582724674; u.sig=1g3gy3Thq5MyFexbtGy2oMIfDW0; s=e319hj7i0g; bid=14636255fdee238fdda1fc0fe35fbb46_jg4ud6kz; Hm_lvt_1db88642e346389874251b5a1eded6e3=1524040463; Hm_lpvt_1db88642e346389874251b5a1eded6e3=1524040514; __utma=1.585935036.1524040465.1524040465.1524040465.1; __utmb=1.5.10.1524040465; __utmc=1; __utmz=1.1524040465.1.1.utmcsr=sogou.com|utmccn=(referral)|utmcmd=referral|utmcct=/link',
    'user_id':'4776750571', 
    },
]        

print(' 已准备好！')

数据引擎：运行于研究
 已准备好！


## 1.设置当前用户

In [2]:
user_id=xueqiu_list[0]['user_id'] 
cookies=xueqiu_list[0]['cookies']
user_info=get_xueqiu_user(user_id,cookies)
user_name=user_info[0]
print(' ID　　',user_id)
print(' 名称　',user_info[0])
print(' 城市　',user_info[1])
print(' 性别　',user_info[2])
print(' 帖子　',user_info[3])
print(' 组合　',user_info[4])
print(' 关注　',user_info[5])
print(' 粉丝　',user_info[6])
print(' 简介　',user_info[7])

NameError: name 'get_xueqiu_cookies' is not defined

## 2.获取贴子

In [5]:
# 断点续爬、更新数据
#user_id：用户id
#cookies：
#resume：断点续爬模式
def update_xueqiu_data(user_id,cookies,resume=False):
    #检查是否已存在数据文件，如存在更新或续爬
    if exists_xueqiu_file(user_id):
        #读取数据
        df=read_xueqiu_data(user_id)
        #如果是续爬模式
        if resume==True:
            #起始位置为最后获取的贴子时间
            update_date=df.date.iloc[-1]
            new_df=get_xueqiu_link(user_id,cookies,update_date,True) 
        else:
            #起始位置为最新贴子时间
            update_date=df.date.iloc[0]
            new_df=get_xueqiu_link(user_id,cookies,update_date,False) 
        if new_df is None:
            print('没有新文章或更新不成功！')
        else:
            if resume==True:
                #拼接新、旧数据，顺序不能反
                df=pd.concat([df,new_df],ignore_index=True)
            else:
                #拼接旧、新数据，顺序不能反
                df=pd.concat([new_df,df],ignore_index=True) 
            #保存到cvs文件
            save_xueqiu_data(user_id,df)
            print(' 共更新：%s 篇文章！'%len(new_df))    
    else:
        #数据文件不存在，首次获取
        update_date=pd.datetime.today().strftime('%Y-%m-%d %H:%M')
        df=get_xueqiu_link(user_id,cookies,update_date,True)  
        if df is None:
            print('获取不成功！')
        else:
            #保存到cvs文件
            save_xueqiu_data(user_id,df)
            print(' 共获取：%s 篇文章！'%len(df))
    
update_xueqiu_data(user_id,cookies,resume=False)

<Response [200]>
没有新文章或更新不成功！


## 4.发送邮件

In [9]:
import sys

#引入自定义库
sys.path.append("/home/jquser/")
from mail import *

#发送某月数据
def send_xueqiu_data(user_id):
    #检查是否已存在数据
    if not exists_xueqiu_file(user_id):
        print('未找到 %s 的雪球发帖数据表！'%user_id)
        return
    #读取数据
    df=read_xueqiu_data(user_id)
    #计算起止年
    start_year=int(df.date.iloc[-1][:4])
    end_year=int(df.date.iloc[0][:4])+1
    #月
    m_list=['01','02','03','04','05','06','07','08','09','10','11','12'] 
    #当前月份
    current_month=pd.datetime.today().strftime('%Y-%m')
    #已发送标记
    issend=False
    #遍历年、月
    for y in range(start_year,end_year):
        for m in m_list:
            month=str(y)+'-'+m
            #判断是否为当月，当月不发送
            if current_month==month:
                continue
            #按照年月筛选    
            temp_df=df[(df.date.str[0:7]==month) & (df.sended==False)]
            temp_df=temp_df.sort('date')
            if len(temp_df)==0:
                continue  
            #生成月邮件正文    
            message=''
            for i in range(len(temp_df)):
                #标题（如果有）
                if str(temp_df.title.iloc[i])!='nan':
                    message+='<h3>%s</h3>'%temp_df.title.iloc[i] 
                #日期
                message+='<p>%s</p>'%temp_df.date.iloc[i]  
                #url
                message+='<p>%s</p>'%temp_df.link.iloc[i] 
                #正文    
                message+='%s</br></br><hr /></br>'%temp_df.text.iloc[i] 
            #邮件主题    
            subjiect='%s年%s月（雪球帖子 %s）'%(y,m,user_name)
            #内容标题
            title='<h2>%s</h2></br><hr />'%(subjiect)
            #邮件内容
            message=title+message
            #去除多余的行
            message=message.replace('<br/><br/>','').replace('</p></br>','</p>').replace('<p><br/></p>','')
            try:
                #发送邮件
                send_html_qqmail(subjiect,message)
                #标记已发送
                if len(temp_df)==1:
                    df.sended.iloc[temp_df.index[0]]=True
                else:
                    df.sended.iloc[temp_df.index[-1]:temp_df.index[0]+1]=True
                issend=True    
                print(' %s 的帖子发送成功！'%(subjiect),end='\r')
                #延时5秒，防止被踢
                time.sleep(3)
            except:
                print(' %s 的帖子发送失败！'%(subjiect),end='\r')
    #如果有发送，则保存            
    if issend:
        save_xueqiu_data(user_id,df)

        
send_xueqiu_data(user_id)  

/opt/conda/lib/python3.4/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## 5.查看数据表

In [7]:
df=read_xueqiu_data(user_id)
# df['sended']=False
# save_xueqiu_data(user_id,df)
df

,date,title,description,text,link,sended
0,2018-04-26 23:57,NaN,再休息三个月。2018的第一次休息。浪费太多时间了。人生苦短，用有限的时间做点有意义的事。八月见。,再休息三个月。2018的第一次休息。<br/><br/>浪费太多时间了。人生苦短，用有限的时...,https://xueqiu.com/4776750571/106113411,True
1,2018-04-26 16:51,NaN,"回复<a href=""http://xueqiu.com/n/yejiangming"" ta...","回复<a href=""http://xueqiu.com/n/yejiangming"" ta...",https://xueqiu.com/4776750571/106077493,True
2,2018-04-26 16:04,NaN,排名一般。,排名一般。,https://xueqiu.com/4776750571/106072614,True
3,2018-04-26 15:52,NaN,"回复<a href=""http://xueqiu.com/n/控计得句计己的狸狸"" targ...","回复<a href=""http://xueqiu.com/n/控计得句计己的狸狸"" targ...",https://xueqiu.com/4776750571/106071401,True
4,2018-04-26 15:25,NaN,"回复<a href=""http://xueqiu.com/n/茫然的笨脚大哥"" target...","回复<a href=""http://xueqiu.com/n/茫然的笨脚大哥"" target...",https://xueqiu.com/4776750571/106069090,True
5,2018-04-26 15:22,NaN,"回复<a href=""http://xueqiu.com/n/又见炊烟1999"" targe...","回复<a href=""http://xueqiu.com/n/又见炊烟1999"" targe...",https://xueqiu.com/4776750571/106068871,True
6,2018-04-26 15:20,NaN,"回复<a href=""http://xueqiu.com/n/Poise"" target=""...","回复<a href=""http://xueqiu.com/n/Poise"" target=""...",https://xueqiu.com/4776750571/106068724,True
7,2018-04-26 14:55,NaN,"回复<a href=""http://xueqiu.com/n/乐视坑爹啊"" target=""...","回复<a href=""http://xueqiu.com/n/乐视坑爹啊"" target=""...",https://xueqiu.com/4776750571/106065784,True
8,2018-04-26 14:42,NaN,忘了说了。其实，太多人并没有经历过真正的深冬。在他们的世界观里，春夏秋冬其实是不完整的。over。,忘了说了。其实，太多人并没有经历过真正的深冬。在他们的世界观里，春夏秋冬其实是不完整的。over。,https://xueqiu.com/4776750571/106064144,True
9,2018-04-26 14:20,NaN,各位，我表达一下自己的一些想法，不对的请指教。 前几天说过，趋势其实很重要。我们不一定按照趋...,<p>各位，我表达一下自己的一些想法，不对的请指教。</p><p><br/></p><p>前...,https://xueqiu.com/4776750571/106061476,True
